## Defining your own classes

### User Defined Types

A **class** is a user-programmed Python type (since Python 2.2!)

It can be defined like:

In [ ]:
class Room(object):
    pass

Or:

In [ ]:
class Room():
    pass

Or:

In [ ]:
class Room:
    pass

What's the difference? Before Python 2.2 a class was distinct from all other Python types, which caused some odd behaviour. To fix this, classes were redefined as user programmed types by extending `object`, e.g., class `room(object)`.

So most Python 2 code will use this syntax as very few people want to use old style python classes. Python 3 has formalised this by removing old-style classes, so they can be defined without extending `object`, or indeed without braces.


Just as with other python types, you use the name of the type as a function to make a variable of that type:

In [ ]:
zero = int()
type(zero)

In [ ]:
myroom = Room()
type(myroom)

In the jargon, we say that an **object** is an **instance** of a particular **class**.

`__main__` is the name of the scope in which top-level code executes, where we've defined the class `Room`.

Once we have an object with a type of our own devising, we can add properties at will:

In [ ]:
myroom.name = "Living"

In [ ]:
myroom.name

The most common use of a class is to allow us to group data into an object in a way that is 
easier to read and understand than organising data into lists and dictionaries.

In [ ]:
myroom.capacity = 3
myroom.occupants = ["Graham", "Eric"]

### Methods

So far, our class doesn't do much!

We define functions **inside** the definition of a class, in order to give them capabilities, just like the methods on built-in
types.

In [ ]:
class Room:
    def overfull(self):
        return len(self.occupants) > self.capacity

In [ ]:
myroom = Room()
myroom.capacity = 3
myroom.occupants = ["Graham", "Eric"]

In [ ]:
myroom.overfull()

In [ ]:
myroom.occupants.append(['TerryG'])

In [ ]:
myroom.occupants.append(['John'])

In [ ]:
myroom.overfull()

When we write methods, we always write the first function argument as `self`, to refer to the object instance itself,
the argument that goes "before the dot".

This is just a convention for this variable name, not a keyword. You could call it something else if you wanted.

### Constructors

Normally, though, we don't want to add data to the class attributes on the fly like that. 
Instead, we define a **constructor** that converts input data into an object. 

In [ ]:
class Room:
    def __init__(self, name, exits, capacity, occupants=[]):
        self.name = name
        self.occupants = occupants  # Note the default argument, occupants start empty
        self.exits = exits
        self.capacity = capacity

    def overfull(self):
        return len(self.occupants) > self.capacity

In [ ]:
living = Room("Living Room", {'north': 'garden'}, 3)

In [ ]:
living.capacity

Methods which begin and end with **two underscores** in their names fulfil special capabilities in Python, such as
constructors.

### Object-oriented design

In building a computer system to model a problem, therefore, we often want to make:

* classes for each *kind of thing* in our system
* methods for each *capability* of that kind
* properties (defined in a constructor) for each *piece of information describing* that kind


For example, the below program might describe our "Maze of Rooms" system:

We define a "Maze" class which can hold rooms:

In [ ]:
class Maze:
    def __init__(self, name):
        self.name = name
        self.rooms = {}

    def add_room(self, room):
        room.maze = self  # The Room needs to know which Maze it is a part of
        self.rooms[room.name] = room

    def occupants(self):
        return [occupant for room in self.rooms.values()
                for occupant in room.occupants.values()]

    def wander(self):
        """Move all the people in a random direction"""
        for occupant in self.occupants():
            occupant.wander()

    def describe(self):
        for room in self.rooms.values():
            room.describe()

    def step(self):
        self.describe()
        print("")
        self.wander()
        print("")

    def simulate(self, steps):
        for _ in range(steps):
            self.step()

And a "Room" class with exits, and people:

In [ ]:
class Room:
    def __init__(self, name, exits, capacity, maze=None):
        self.maze = maze
        self.name = name
        self.occupants = {}  # Note the default argument, occupants start empty
        self.exits = exits  # Should be a dictionary from directions to room names
        self.capacity = capacity

    def has_space(self):
        return len(self.occupants) < self.capacity

    def available_exits(self):
        return [exit for exit, target in self.exits.items()
                if self.maze.rooms[target].has_space()]

    def random_valid_exit(self):
        import random
        if not self.available_exits():
            return None
        return random.choice(self.available_exits())

    def destination(self, exit):
        return self.maze.rooms[self.exits[exit]]

    def add_occupant(self, occupant):
        occupant.room = self  # The person needs to know which room it is in
        self.occupants[occupant.name] = occupant

    def delete_occupant(self, occupant):
        del self.occupants[occupant.name]

    def describe(self):
        if self.occupants:
            print(f"{self.name}: " + " ".join(self.occupants.keys()))

We define a "Person" class for room occupants:

In [ ]:
class Person:
    def __init__(self, name, room=None):
        self.name = name

    def use(self, exit):
        self.room.delete_occupant(self)
        destination = self.room.destination(exit)
        destination.add_occupant(self)
        print("{some} goes {action} to the {where}".format(some=self.name,
                                                           action=exit,
                                                           where=destination.name))

    def wander(self):
        exit = self.room.random_valid_exit()
        if exit:
            self.use(exit)

And we use these classes to define our people, rooms, and their relationships:

In [ ]:
graham = Person('Graham')
eric = Person('Eric')
terryg = Person('TerryG')
john = Person('John')

In [ ]:
living = Room('livingroom', {'outside': 'garden',
                             'upstairs': 'bedroom', 'north': 'kitchen'}, 2)
kitchen = Room('kitchen', {'south': 'livingroom'}, 1)
garden = Room('garden', {'inside': 'livingroom'}, 3)
bedroom = Room('bedroom', {'jump': 'garden', 'downstairs': 'livingroom'}, 1)

In [ ]:
house = Maze('My House')

In [ ]:
for room in [living, kitchen, garden, bedroom]:
    house.add_room(room)

In [ ]:
living.add_occupant(graham)

In [ ]:
garden.add_occupant(eric)
garden.add_occupant(terryg)

In [ ]:
bedroom.add_occupant(john)

And we can run a "simulation" of our model:

In [ ]:
house.simulate(3)

### Object oriented design

There are many choices for how to design programs to do this. Another choice would be to separately define exits as a different class from rooms. This way, 
we can use arrays instead of dictionaries, but we have to first define all our rooms, then define all our exits.

In [ ]:
class Maze:
    def __init__(self, name):
        self.name = name
        self.rooms = []
        self.occupants = []

    def add_room(self, name, capacity):
        result = Room(name, capacity)
        self.rooms.append(result)
        return result

    def add_exit(self, name, source, target, reverse=None):
        source.add_exit(name, target)
        if reverse:
            target.add_exit(reverse, source)

    def add_occupant(self, name, room):
        self.occupants.append(Person(name, room))
        room.occupancy += 1

    def wander(self):
        "Move all the people in a random direction"
        for occupant in self.occupants:
            occupant.wander()

    def describe(self):
        for occupant in self.occupants:
            occupant.describe()

    def step(self):
        house.describe()
        print("")
        house.wander()
        print("")

    def simulate(self, steps):
        for _ in range(steps):
            self.step()

In [ ]:
class Room:
    def __init__(self, name, capacity):
        self.name = name
        self.capacity = capacity
        self.occupancy = 0
        self.exits = []

    def has_space(self):
        return self.occupancy < self.capacity

    def available_exits(self):
        return [exit for exit in self.exits if exit.valid()]

    def random_valid_exit(self):
        import random
        if not self.available_exits():
            return None
        return random.choice(self.available_exits())

    def add_exit(self, name, target):
        self.exits.append(Exit(name, target))

In [ ]:
class Person:
    def __init__(self, name, room=None):
        self.name = name
        self.room = room

    def use(self, exit):
        self.room.occupancy -= 1
        destination = exit.target
        destination.occupancy += 1
        self.room = destination
        print("{some} goes {action} to the {where}".format(some=self.name,
                                                           action=exit.name,
                                                           where=destination.name))

    def wander(self):
        exit = self.room.random_valid_exit()
        if exit:
            self.use(exit)

    def describe(self):
        print("{who} is in the {where}".format(who=self.name,
                                               where=self.room.name))

In [ ]:
class Exit:
    def __init__(self, name, target):
        self.name = name
        self.target = target

    def valid(self):
        return self.target.has_space()

In [ ]:
house = Maze('My New House')

In [ ]:
living = house.add_room('livingroom', 2)
bed = house.add_room('bedroom', 1)
garden = house.add_room('garden', 3)
kitchen = house.add_room('kitchen', 1)

In [ ]:
house.add_exit('north', living, kitchen, 'south')

In [ ]:
house.add_exit('upstairs', living, bed, 'downstairs')

In [ ]:
house.add_exit('outside', living, garden, 'inside')

In [ ]:
house.add_exit('jump', bed, garden)

In [ ]:
house.add_occupant('Graham', living)
house.add_occupant('Eric', garden)
house.add_occupant('TerryJ', bed)
house.add_occupant('John', garden)

In [ ]:
house.simulate(3)

This is a huge topic, about which many books have been written. The differences between these two designs are important, and will have long-term consequences for the project. That is the how we start to think about **software engineering**, as opposed to learning to program, and is an important part of this course.

### Exercise: Your own solution

Compare the two solutions above. Discuss with a partner which you like better, and why. Then, starting from scratch, design your own. What choices did you make that are different from mine?